In [ ]:
import sys
sys.path.append('/Users/zhengj10/Python/Modules')

import h5py
import os
import numpy as np
from skimage.io import imsave
import matplotlib.pyplot as plt
import yaml
import pandas as pd
import PatcherBot as pb
from glob import glob
import seaborn as sns


### Load data

In [ ]:
# imgPath = '/Users/zhengj10/Desktop/Temp/Ben/'
sumPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/summary'
# imgPath = '/Volumes/genie/BenArthur/patch-screen/bot'
# rawPath = '/Users/zhengj10/Desktop/'
# rawPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw_dead_sensor'
rawPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw'
os.path.exists(rawPath)


In [ ]:
# datePath = pb.getPathInfo(rawPath)
datePath = ['2020.07.22_000','2020.07.23_000']
datePath


In [ ]:
df = pd.read_pickle(sumPath + '/summary_sel.pkl')
print(len(df))
df


In [ ]:
def get_dff(trace):
    dff = []
    ref = np.mean(trace[50:120,:,:],axis=0)
    fb = np.min(ref)
    crop = ref[234:274,:]
    max_val = np.max(crop)
    act_pix = crop > (fb + (max_val-fb)*0.67)
    f0 = np.sum(act_pix * crop) / np.sum(act_pix)
    for frame in range(trace.shape[0]):
        dff_single = ((np.sum(trace[frame, 234:274, :]*act_pix)/np.sum(act_pix)) - f0)/(f0-fb)
        dff.append(dff_single)
    dff = np.mean(dff[950:1050])
    return dff, f0


In [ ]:
columns = ['Date','Plate','Well','Cell','Mutation','Transfected_date','Leak','Ra','dFF','F0','Leak_min','Ra_max']
df = pd.DataFrame(columns = columns)
df


In [ ]:
data_all={}
for param in(df.columns):
    data_all[param] = []
for date in datePath:
    platePath = pb.getPathInfo(os.path.join(rawPath, date))
    for plate in platePath:
        idxFile = os.path.join(rawPath, date, plate, '.index')
        a = yaml.load(open(idxFile), Loader=yaml.FullLoader)
        mutation = a['.']['mutation_1']
#         if date == '2020.02.03_000':
#             mutation = '421.1(em:545/20)'
#         else:
#             mutation = '421.1(em:545/40)'
        note = a['.']['notes'][0:6]
        cellPath = pb.getPathInfo(os.path.join(rawPath, date, plate))
        for cell in(cellPath):
            if not os.path.exists(os.path.join(rawPath, date, plate, cell, 'camera_clamp1_000')):
                continue
            print(os.path.join(rawPath, date, plate, cell))
            data_all['Date'].append(date)
            data_all['Plate'].append(plate)
            data_all['Well'].append('well_1')
            data_all['Cell'].append(cell)
            data_all['Mutation'].append(mutation)
            data_all['Transfected_date'].append(note[:6])
            leaks = []
            Ra_s = [] # List of Ra
            F0 = []
            dff_list = []

            for trial in range(9):
                dataFile = os.path.join(rawPath,date,plate,cell,'camera_clamp1_000',str(trial).zfill(3),'Camera','frames.ma')

                data = h5py.File(dataFile)
                imgData = data['data'][()]
                dff_single, f0_single = get_dff(imgData)
                ivFile = os.path.join(rawPath,date,plate,cell,'camera_clamp1_000',str(trial).zfill(3),'Clamp1.ma')
                leak = pb.get_leak(ivFile)
                Ra = pb.get_Ra(ivFile,trial)
                leaks.append(leak)
                Ra_s.append(Ra)
                F0.append(f0_single)
                dff_list.append(dff_single)


            data_all['F0'].append(F0)
            data_all['Leak'].append(leaks)
            data_all['Ra'].append(Ra_s)
            data_all['Leak_min'].append(np.min(leaks))
            data_all['Ra_max'].append(np.max(Ra_s))


            data_all['dFF'].append(dff_list)
print('Done!')


In [ ]:
df = pd.DataFrame(data_all, columns = list(df)) # columns specification keeps the order of columns; index specification makes sure dict keys are the columns 
df

### Plot

- Filter data

In [ ]:
threshold = {
            'leak':-100,
            'Ra':30
        }


In [ ]:
df = df[(df.Leak_min>threshold['leak'])&(df.Ra_max<threshold['Ra'])]
# pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_colwidth', 80)
print(len(df))
# df


- Plot dFF etc for each voltage step

In [ ]:
sensors = df.Mutation.unique()
# sensors = ['376.20950','485.20950']
sensors


In [ ]:
# %matplotlib tk # plot in a new window
import unicodedata
import seaborn as sns
sns.set()
sns.set_style(style='whitegrid')

fig = plt.figure(figsize=[5.3,5])
params = ['dFF']
x_label = 'mV'
y_label = [unicodedata.lookup("GREEK CAPITAL LETTER delta") + 'F/F']

for i in range(len(params)):
    ax = plt.subplot(1,1,i+1)
    for sensor in sensors: # Manually pick up sensors to plot

    
        df_sensor = df[df.Mutation == sensor]
        param_mean = np.nanmean(df_sensor[params[i]].tolist(), axis=0)
        param_sem = np.nanstd(df_sensor[params[i]].tolist(), axis=0)/(len(df_sensor)-1)
        x = np.linspace(-100,60,9)
        ax.errorbar(x, param_mean, yerr = param_sem, fmt = 'o-', label = sensor + ' (%d)' %len(df_sensor))
    ax.legend()
    ax.set_xlabel(x_label)
#     ax.set_ylim([-0.3,0.3])
    ax.set_ylabel(y_label[i])
#     ax.set_title(params[i])



plt.setp(ax, xticks=x)
plt.tight_layout()

handles, labels = ax.get_legend_handles_labels() # Following 3 lines remove the errorbars from legend
handles = [h[0] for h in handles]
ax.legend(handles, labels)
ax.set_ylim([-0.1,0.05])
# fig.savefig('/Users/zhengj10/Desktop/dead_voltron_positron.png',dpi=1200)
plt.show()


In [ ]:
df_sel = df[df['Plate'] == 'plate_001']
df_sel.index = np.arange(len(df_sel))
df_sel

In [ ]:
fig = plt.figure(figsize=[6,6])
for i,r in df_sel.iterrows():
    imgFile = os.path.join(rawPath, r['Date'], r['Plate'], r['Cell'], 'fluor_image_000.tif')
    imgFile = tff.TiffFile(imgFile)
    imgFile = imgFile.asarray()[193:321,193:321]
    ax = plt.subplot(3,3,i+1)
    ax.imshow(imgFile, cmap = 'gray')
    if i==0:
        ax.plot([95,115],[115,115],color='w')
    ax.set_xticks([]) 
    ax.set_yticks([])
# fig.suptitle(r['Mutation'])
plt.tight_layout()
# fig.savefig('/Users/zhengj10/Desktop/421.4494.png',dpi=1200)
plt.show()


In [ ]:
df['Date'][0][:10]

In [ ]:
import tifffile as tff
fig = plt.figure(figsize = [12,9.5])

for date in datePath:
    platePath = pb.getPathInfo(os.path.join(rawPath, date))
    i = 1
    for plate in platePath:
        if plate == 'plate_000':
            print(plate)
            idxFile = os.path.join(rawPath, date, plate, '.index')
            a = yaml.load(open(idxFile), Loader=yaml.FullLoader)
            mutation = a['.']['mutation_1']

            cellPath = pb.getPathInfo(os.path.join(rawPath, date, plate))



            for cell in cellPath:

                imgFile = os.path.join(rawPath, date, plate, cell, 'fluor_image_000.tif')
                if not os.path.exists(imgFile):
                    continue
                imgFile = tff.TiffFile(imgFile)
                imgFile = imgFile.asarray()[193:321,193:321]
                ax = plt.subplot(3,4,i)
                ax.imshow(imgFile, cmap = 'gray')
    #                 ax.set_title(mutation)
                i += 1






                ax.set_xticks([]) 
                ax.set_yticks([]) 

plt.tight_layout()
# plt.savefig('/Users/zhengj10/Desktop/focus_plane_128x128.png')
plt.show()



In [ ]:
df

In [ ]:
sns.set_style(style='whitegrid')

fig = plt.figure(figsize=[2,2.5])
params = ['F0']
f0_mean = []
f0_sem = []
for i in range(len(params)):
    ax = plt.subplot(1,1,i+1)
    for sensor in sensors: # Manually pick up sensors to plot

    
        df_sensor = df[df.Mutation == sensor]
        param_mean = np.nanmean(df_sensor[params[i]].tolist(), axis=0)[0]
        param_sem = np.nanstd(df_sensor[params[i]].tolist(), axis=0)[0]/(len(df_sensor)-1)
        f0_mean.append(param_mean)
        f0_sem.append(param_sem)
        
x = np.arange(1,len(sensors)+1)
        
ax.bar(x, f0_mean, yerr = f0_sem)
ax.set_ylabel('F0')
plt.xticks(x, sensors, rotation=30, ha = 'right')
# plt.setp(ax, xticks=x)
plt.tight_layout()

# fig.savefig('/Users/zhengj10/Desktop/Dead_Voltron_positron_f0.png',dpi=1200)
plt.show()
